In [1]:
import lhapdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt


# Global Constants
M1_test=np.sqrt(0.8)
Kp2A=0.57
Pp2A=0.12

ee=1
eU=2/3
eUbar=-2/3
eD=-1/3
eDbar=1/3
eS=-1/3
eSbar=1/3

AlphaU_test=1.0
BetaU_test=6.6
AlphaD_test=1.9
BetaD_test=10
AlphaS_test=0
BetaS_test=0

NU_test=0.18
NUbar_test=-0.01
ND_test=-0.52
NDbar_test=-0.06
NS_test=0
NSbar_test=0

# Here the data sets are listed and collected into and Array called "DataFilesArray"
Dat1='Data/HERMES_p_2009.csv'
Dat2='Data/HERMES_p_2020.csv'
Dat3='Data/COMPASS_d_2009.csv'
Dat4='Data/COMPASS_p_2015.csv'
DataFilesArray=[Dat1,Dat2,Dat3,Dat4]
PDFdataset = lhapdf.mkPDF("JAM19PDF_proton_nlo")
#PDFdataset = lhapdf.mkPDF("CT10nnlo")
FF_pion_dataset=["JAM19FF_pion_nlo"]
FF_kaon_dataset=["JAM19FF_kaon_nlo"]

In [2]:
def hadarray(filename):
    tempdf=pd.read_csv(filename)
    temphad_data=tempdf["hadron"]
    temphad=temphad_data.dropna().unique()
    refined_had_array=[]
    for i in range(0,len(temphad)):
        if((temphad[i]=="pi+") or (temphad[i]=="pi-") or (temphad[i]=="pi0") or (temphad[i]=="k+") or (temphad[i]=="k-")):
            refined_had_array.append(temphad[i])
    return refined_had_array

#print(hadarray(Datafile))

def dataslice(filename,Had,Var):
    tempdf=pd.read_csv(filename)
    temp_slice=tempdf[(tempdf["hadron"]==Had)&(tempdf["1D_dependence"]==Var)]
    tempQ2=np.array(temp_slice["Q2"])
    tempX=np.array(temp_slice["x"])
    tempZ=np.array(temp_slice["z"])
    tempPHT=np.array(temp_slice["phT"])
    tempSiv=np.array(temp_slice["Siv"])
    temperrSiv=np.array(temp_slice["tot_err"])
    return tempQ2,tempX,tempZ,tempPHT,tempSiv,temperrSiv

def ks2Avg(m1,kperp2Avg):
    test_ks2Avg=((m1**2)*kperp2Avg)/((m1**2)+kperp2Avg)
    return test_ks2Avg

def A0(z,pht,m1,kperp2Avg,pperp2Avg,eCharg):
    tempA0part1=(((z**2)*kperp2Avg+pperp2Avg)*((ks2Avg(m1,kperp2Avg))**2))/((((z**2)*(ks2Avg(m1,kperp2Avg))+pperp2Avg)**2)*kperp2Avg)
    tempA0part21=(pht**2)*(z**2)*(ks2Avg(m1,kperp2Avg) - kperp2Avg)
    tempA0part22=((z**2)*(ks2Avg(m1,kperp2Avg))+pperp2Avg)*((z**2)*kperp2Avg+pperp2Avg)
    tempA0part2=np.exp(-tempA0part21/tempA0part22)
    tempA0part3=(np.sqrt(2*eCharg))*z*pht/m1
    tempA0=tempA0part1*tempA0part2*tempA0part3
    return tempA0

def NNq(x,Nq,aq,bq):
    tempNNq = Nq*(x**aq)*((1-x)**(bq))*((aq+bq)**(aq+bq))/((aq**aq)*(bq**bq))
    return tempNNq

def NNqbar(x,Nqbar):
    tempNNqbar = Nqbar
    return tempNNqbar

def xFxQ2(dataset,flavor,x,QQ):
    temp_parton_dist_x=(dataset.xfxQ2(flavor, x, QQ))
    return temp_parton_dist_x

def zFzQ(dataset,flavor,zz,QQ):
    # Here "0" represents the central values from the girds
    temp_zD1=lhapdf.mkPDF(dataset[0], 0)
    zD1_vec=(temp_zD1.xfxQ2(flavor,zz,QQ))
    return zD1_vec

def Asymmetry(QQ,x,z,pht,m1,Nq,aq,bq,Nqbar,eq,eqbar,lhaqID,lhaqbarID):
    kperp2Avg=Kp2A
    pperpAvg=Pp2A
    eCharg=ee
    if(lhaqID==3):
        Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqID,z,QQ)
        dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqID,z,QQ)
        dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        tempNumerator = Ucontribution1 + dbarcontribution1
        tempDenominator = Ucontribution2 + dbarcontribution2
        tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    elif(lhaqbarID==-3):
        Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqbarID,z,QQ)
        Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqbarID,z,QQ)
        tempNumerator = Ucontribution1 + dbarcontribution1
        tempDenominator = Ucontribution2 + dbarcontribution2
        tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    else:
        Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        tempNumerator = Ucontribution1 + dbarcontribution1
        tempDenominator = Ucontribution2 + dbarcontribution2
        tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    return tempASivPiP
    
### The following segment was assuming the iso-spin symmetry in Fragmentation Functions
    
# def Asymmetry(QQ,x,z,pht,m1,Nq,aq,bq,Nqbar,eq,eqbar,lhaqID,lhaqbarID):
#     kperp2Avg=Kp2A
#     pperpAvg=Pp2A
#     eCharg=ee
#     Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
#     dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
#     Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
#     dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
#     tempNumerator = Ucontribution1 + dbarcontribution1
#     tempDenominator = Ucontribution2 + dbarcontribution2
#     tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
#     return tempASivPiP


In [3]:
## ** two stars represents a dictionary
def ASivFitHadron(hadron,KV,**parms):    
    if(hadron=="pi+"):
        Qflag=2
        AniQflag=-1
        eQ=eU
        eQbar=eDbar
        m1= parms["m1"]
        NQ = parms["Nu"]
        alphaQ = parms["alphau"]
        betaQ = parms["betau"]
        NQbar = parms["Ndbar"]
    elif(hadron=="pi-"):
        Qflag=1
        AniQflag=-2
        eQ=eD
        eQbar=eUbar 
        m1= parms["m1"]
        NQ = parms["Nd"]
        alphaQ = parms["alphad"]
        betaQ = parms["betad"]
        NQbar = parms["Nubar"]
    elif(hadron=="pi0"):
        Qflag=1
        AniQflag=-1
        eQ=eU
        eQbar=eUbar
        m1= parms["m1"]
        NQ = parms["Nu"]
        alphaQ = parms["alphau"]
        betaQ = parms["betau"]
        NQbar = parms["Nubar"]
    elif(hadron=="k+"):
        Qflag=2
        AniQflag=-3
        eQ=eU
        eQbar=eSbar
        m1= parms["m1"]
        NQ = parms["Nu"]
        alphaQ = parms["alphau"]
        betaQ = parms["betau"]
        NQbar = parms["Nsbar"]
    elif(hadron=="k-"):
        Qflag=3
        AniQflag=-2
        eQ=eS
        eQbar=eUbar
        m1= parms["m1"]
        NQ = parms["Ns"]
        alphaQ = parms["alphas"]
        betaQ = parms["betas"]
        NQbar = parms["Nubar"]
    ################
    QQ,x,z,pht=KV
    array_size=len(x)
    tempASivHad_val=[]
    for i in range(0,array_size):
        tempASivHad=Asymmetry(QQ[i],x[i],z[i],pht[i],m1,NQ,alphaQ,betaQ,NQbar,eQ,eQbar,Qflag,AniQflag)
        tempASivHad_val.append(tempASivHad)
    return tempASivHad_val


In [4]:
def Kin_hadron(datfile,hadron):
    ##### Q2 ################
    tempQ2_x=np.array(dataslice(datfile,hadron,"x")[0])
    tempQ2_z=np.array(dataslice(datfile,hadron,"z")[0])
    tempQ2_phT=np.array(dataslice(datfile,hadron,"phT")[0])
    tempQ2=np.concatenate((tempQ2_x,tempQ2_z,tempQ2_phT))
    ##### X ################
    tempX_x=np.array(dataslice(datfile,hadron,"x")[1])
    tempX_z=np.array(dataslice(datfile,hadron,"z")[1])
    tempX_phT=np.array(dataslice(datfile,hadron,"phT")[1])
    tempX=np.concatenate((tempX_x,tempX_z,tempX_phT))
    ##### Z ################
    tempZ_x=np.array(dataslice(datfile,hadron,"x")[2])
    tempZ_z=np.array(dataslice(datfile,hadron,"z")[2])
    tempZ_phT=np.array(dataslice(datfile,hadron,"phT")[2])
    tempZ=np.concatenate((tempZ_x,tempZ_z,tempZ_phT))
    ##### phT ################
    tempphT_x=np.array(dataslice(datfile,hadron,"x")[3])
    tempphT_z=np.array(dataslice(datfile,hadron,"z")[3])
    tempphT_phT=np.array(dataslice(datfile,hadron,"phT")[3])
    tempphT=np.concatenate((tempphT_x,tempphT_z,tempphT_phT))
    return tempQ2,tempX,tempZ,tempphT


def Kin_Had(datfile):
    had_len=len(hadarray(datfile))
    temHads=hadarray(datfile)
    temp_kin=[]
    for i in range(0,had_len):
        temp_kin.append(Kin_hadron(datfile,temHads[i]))        
    return temp_kin

#print(len(Kin_Had(Datafile)))
#print(Kin_Had(Datafile)[3])


#### Sivers values

def ASiv_data(datfile,hadron):
    ##### Asy ################
    tempAsy_x=np.array(dataslice(datfile,hadron,"x")[4])
    tempAsy_z=np.array(dataslice(datfile,hadron,"z")[4])
    tempAsy_phT=np.array(dataslice(datfile,hadron,"phT")[4])
    tempAsy=np.concatenate((tempAsy_x,tempAsy_z,tempAsy_phT))
    ##### err ################
    tempAsyErr_x=np.array(dataslice(datfile,hadron,"x")[5])
    tempAsyErr_z=np.array(dataslice(datfile,hadron,"z")[5])
    tempAsyErr_phT=np.array(dataslice(datfile,hadron,"phT")[5])
    tempAsyErr=np.concatenate((tempAsyErr_x,tempAsyErr_z,tempAsyErr_phT))
    return tempAsy,tempAsyErr

def ASiv_Val(datfile):
    had_len=len(hadarray(datfile))
    temHads=hadarray(datfile)
    temp_SivData=[]
    for i in range(0,had_len):
        temp_SivData.append(ASiv_data(datfile,temHads[i])[0])        
    return temp_SivData

def ASiv_Err(datfile):
    had_len=len(hadarray(datfile))
    temHads=hadarray(datfile)
    temp_SivData=[]
    for i in range(0,had_len):
        temp_SivData.append(ASiv_data(datfile,temHads[i])[1])        
    return temp_SivData

#print(np.concatenate(ASiv_Val(Datafile), axis=None))
#print(np.concatenate(ASiv_Err(Datafile), axis=None))

In [5]:
#ASivFitHadron("pi+",Kin_hadron(Datafile,"pi+"),m1=1,Nu=1,alphau=1,betau=1,Ndbar=1)

In [6]:
##### This function will calculate the theory values for each data set

def totalfitDataSet(datfile,m1,Nu,alphau,betau,Ndbar,Nd,alphad,betad,Nubar,Ns,alphas,betas,Nsbar):
    had_len=len(hadarray(datfile))
    temHads=hadarray(datfile)
    fittot=[]
    for i in range(0,had_len):
        if temHads[i]=="pi+":
            tempfit=ASivFitHadron("pi+",Kin_hadron(datfile,"pi+"),m1=m1,Nu=Nu,alphau=alphau,betau=betau,Ndbar=Ndbar)
            fittot.append(tempfit)
        elif temHads[i]=="pi-":
            tempfit=ASivFitHadron("pi-",Kin_hadron(datfile,"pi-"),m1=m1,Nd=Nd,alphad=alphad,betad=betad,Nubar=Nubar)
            fittot.append(tempfit)
        elif temHads[i]=="pi0":
            tempfit=ASivFitHadron("pi0",Kin_hadron(datfile,"pi0"),m1=m1,Nu=Nu,alphau=alphau,betau=betau,Nubar=Nubar)
            fittot.append(tempfit)
        elif temHads[i]=="k+":
            tempfit=ASivFitHadron("k+",Kin_hadron(datfile,"k+"),m1=m1,Nu=Nu,alphau=alphau,betau=betau,Nsbar=Nsbar)
            fittot.append(tempfit)
        elif temHads[i]=="k-":
            tempfit=ASivFitHadron("k-",Kin_hadron(datfile,"k-"),m1=m1,Ns=Ns,alphas=alphas,betas=betas,Nubar=Nubar)
            fittot.append(tempfit)
    return np.concatenate((fittot), axis=None)


### This function will combine all the data sets into a single array

def totalfitfunc(datfilesarray,m1,Nu,alphau,betau,Ndbar,Nd,alphad,betad,Nubar,Ns,alphas,betas,Nsbar):
    datfilesnum=len(datfilesarray)
    temptotal=[]
    for i in range(0,datfilesnum):
        temptotal.append(totalfitDataSet(datfilesarray[i],m1,Nu,alphau,betau,Ndbar,Nd,alphad,betad,Nubar,Ns,alphas,betas,Nsbar))
    return np.concatenate((temptotal), axis=None)


### This function collects all data and combine together into one array
    
def SiversVals(datafilesarray):
    datfilesnum=len(datafilesarray)
    tempSiv=[]
    for i in range(0,datfilesnum):
        tempSiv.append(ASiv_Val(datafilesarray[i]))
    tempflatSiv=np.concatenate((tempSiv), axis=None)
    return np.concatenate((tempflatSiv), axis=None)

def SiversErrVals(datafilesarray):
    datfilesnum=len(datafilesarray)
    tempSivErr=[]
    for i in range(0,datfilesnum):
        tempSivErr.append(ASiv_Err(datafilesarray[i]))
    tempflatSivErr=np.concatenate((tempSivErr), axis=None)
    return np.concatenate((tempflatSivErr), axis=None)



In [7]:
#totalfit(Datafile,m1=1,Nu=1,alphau=1,betau=1,Ndbar=1,Nd=1,alphad=1,betad=1,Nubar=1,Ns=1,alphas=1,betas=1,Nsbar=1)
#SiversErrVals(DataFilesArray)
#totalfitfunc(DataFilesArray,m1=1,Nu=1,alphau=1,betau=1,Ndbar=1,Nd=1,alphad=1,betad=1,Nubar=1,Ns=1,alphas=1,betas=1,Nsbar=1)

In [8]:
### This is where I'm trying to do the global fit

p0=1,1,1,1,1,1,1,1,1,1,1,1,1
result, result_cov=opt.curve_fit(totalfitfunc,DataFilesArray,SiversVals(DataFilesArray),p0,sigma=SiversErrVals(DataFilesArray),method='lm')

ValueError: could not convert string to float: 'Data/HERMES_p_2009.csv'

In [ ]:
### This function can be used to make a plot for a given data set.hadron,dependence
dep="x"

def PlotSivHad(datfile,hadron,dependence):
    data_points=len(dataslice(datfile,hadron,dependence)[0])
    temp_kinematics=np.array(dataslice(datfile,hadron,dependence))
    if(dependence=="x"):
        dep_index=1
    elif(dependence=="z"):
        dep_index=2
    elif(dependence=="phT"):
        dep_index=3
    tempQ=temp_kinematics[0]
    tempX=temp_kinematics[1]
    tempZ=temp_kinematics[2]
    tempphT=temp_kinematics[3]
    temp_exp=temp_kinematics[4]
    temp_sigma=temp_kinematics[5]
    temp_theory=ASivFitHadron("pi+",(tempQ,tempX,tempZ,tempphT),m1=result[0],Nu=result[1],alphau=result[2],betau=result[3],Ndbar=result[4])
    plt.plot(temp_kinematics[dep_index],temp_theory,'red')
    plt.errorbar(temp_kinematics[dep_index],temp_kinematics[4],temp_kinematics[5],fmt='o',color='blue')

PlotSivHad(DataFilesArray[1],"pi+",dep)